In [2]:
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bridge_fe_models import *
from scipy import signal

F_SIM = 1024
import ajf_noise_model

#ajf_noise_model.initialise(req_Fs=F_SIM)

import ajf_plts

SEED = ajf_plts.get_seed("5_demo_simulations.ipynb")

plt.style.use(["./ajf_plts/base.mplstyle", "./ajf_plts/legend_frame.mplstyle"])

# Load input vehicle and temperature data

In [3]:
in_df = pd.read_parquet("./vehicle_loads.parquet")

# Load FE model

In [4]:
beam = construct_model_LB()

In [5]:
row = in_df.loc[1]

In [7]:
row

seq_month          0.000000e+00
year               0.000000e+00
month              0.000000e+00
day                1.000000e+00
hour               4.000000e+00
minute             4.300000e+01
dayofweek          6.000000e+00
air_temperature    3.555000e+00
freq               5.795990e+00
E_val              2.892656e+10
AxleCount          3.000000e+00
W1                 1.186605e+05
W2                 1.137571e+05
W3                 6.864655e+04
W4                 0.000000e+00
W5                 0.000000e+00
W6                 0.000000e+00
S0                 0.000000e+00
S1                 4.500000e+00
S2                 5.850000e+00
S3                          NaN
S4                          NaN
S5                          NaN
speed              2.400000e+01
Name: 1, dtype: float64

In [6]:
beam.E = row.E_val
beam.update_model(w_1=2.0 * np.pi * row.freq)

x_dam = LB_span_length / 3.0
deltas = [0.0, 0.2]


times = []
disps = []
stat_disps = []

for d in deltas:
    beam.reset_crack_damage()
    beam.add_crack_damage(x_dam, d)
    beam.update_model(w_1=2.0 * np.pi * row.freq)

    _, stat_disp = beam.perform_static_sim(
        row.ax_weights,
        row.ax_spacing,
        force_step=row.speed / F_SIM,
        pad_steps=1 * int(F_SIM),
    )

    time, disp, _, _ = beam.perform_dynamic_sim(
        row['W1','W2','W3','W4','W5'],
        row.ax_spacing,
        row.speed,
        time_step=1.0 / F_SIM,
        pad_steps=1 * int(F_SIM),
    )
    times.append(time)
    disps.append(disp)
    stat_disps.append(stat_disp)

AttributeError: 'Series' object has no attribute 'ax_weights'

In [ ]:
%matplotlib widget

figsize = (ajf_plts.text_width_inches, 4.5 * ajf_plts.fig_height_inches)
fig, axes = plt.subplots(figsize=figsize, nrows=4, ncols=2)

axes = axes.ravel()

dofs = [beam.x_pos_to_rot_dof(x) for x in [0, LB_span_length]]

for i, dof in enumerate(dofs):
    for j, d in enumerate(deltas):
        rot = disps[j][dof]
        # PAD + ADD NOISE
        noisy = ajf_noise_model.add_noise(s=rot, use_g=True)

        # FILTER
        sos = signal.iirfilter(
            N=6,
            Wn=1.0,  # [0.05, 1.0],
            btype="lowpass",
            ftype="butter",
            analog=False,
            fs=F_SIM,
            output="sos",
        )
        filt = signal.sosfiltfilt(sos, noisy)
        filt -= np.quantile(filt, 0.99 if i % 2 else 0.05)

        axes[i].plot(times[j], 1e6 * rot, label=rf"$\delta={d:1.1f}$")
        axes[i + 2].plot(times[j], 1e3 * noisy, label=rf"$\delta={d:1.1f}$")
        axes[i + 4].plot(
            times[j],
            1e6 * filt,
            label=rf"$\delta={d:1.1f}$",
        )
        axes[i + 6].plot(
            times[j][:-1], 1e6 * stat_disps[j][dof], label=rf"$\delta={d:1.1f}$"
        )

for ax, loc in zip(axes, 4 * ["LHS Abutment", "RHS Abutment"]):
    ax.set_xlabel("Time / s")
    ax.set_ylabel(r"Rotation / \unit{\micro\radian}")
    ax.set_title(loc)
    ax.legend(handlelength=1.25, handletextpad=0.4, borderpad=0.2)

ajf_plts.caption_axes(axes)
fig.tight_layout()

# Gracehill

In [ ]:
beam = construct_model_GH()

In [ ]:
beam.E = row.E_val
beam.update_model(w_1=2.0 * np.pi * row.freq)

x_dam = GH_side_span_length + GH_center_span_length / 3.0
deltas = [0.0, 0.2]


times = []
disps = []
stat_disps = []

for d in deltas:
    beam.reset_crack_damage()
    beam.add_crack_damage(x_dam, d)
    beam.update_model(w_1=2.0 * np.pi * row.freq)

    _, stat_disp = beam.perform_static_sim(
        row.ax_weights,
        row.ax_spacing,
        force_step=row.speed / F_SIM,
        pad_steps=1 * int(F_SIM),
    )

    time, disp, _, _ = beam.perform_dynamic_sim(
        row.ax_weights,
        row.ax_spacing,
        row.speed,
        time_step=1.0 / F_SIM,
        pad_steps=1 * int(F_SIM),
    )
    times.append(time)
    disps.append(disp)
    stat_disps.append(stat_disp)

In [ ]:
%matplotlib widget

figsize = (ajf_plts.text_width_inches, 4.5 * ajf_plts.fig_height_inches)
fig, axes = plt.subplots(figsize=figsize, nrows=4, ncols=2)

axes = axes.ravel()

dofs = [
    beam.x_pos_to_rot_dof(x)
    for x in [GH_side_span_length, GH_side_span_length + GH_center_span_length]
]

for i, dof in enumerate(dofs):
    for j, d in enumerate(deltas):
        rot = disps[j][dof]
        # PAD + ADD NOISE
        noisy = ajf_noise_model.add_noise(s=rot, use_g=True)

        # FILTER
        sos = signal.iirfilter(
            N=6,
            Wn=1.0,  # [0.05, 1.0],
            btype="lowpass",
            ftype="butter",
            analog=False,
            fs=F_SIM,
            output="sos",
        )
        filt = signal.sosfiltfilt(sos, noisy)
        filt -= np.quantile(filt, 0.99 if i % 2 else 0.05)

        axes[i].plot(times[j], 1e6 * rot, label=rf"$\delta={d:1.1f}$")
        axes[i + 2].plot(times[j], 1e3 * noisy, label=rf"$\delta={d:1.1f}$")
        axes[i + 4].plot(
            times[j],
            1e6 * filt,
            label=rf"$\delta={d:1.1f}$",
        )
        axes[i + 6].plot(
            times[j][:-1], 1e6 * stat_disps[j][dof], label=rf"$\delta={d:1.1f}$"
        )

for ax, loc in zip(axes, 4 * ["L Pier", "R Pier"]):
    ax.set_xlabel("Time / s")
    ax.set_ylabel(r"Rotation / \unit{\micro\radian}")
    ax.set_title(loc)
    ax.legend(handlelength=1.25, handletextpad=0.4, borderpad=0.2)

ajf_plts.caption_axes(axes)
fig.tight_layout()